# Example of an end to end ML project

Source: Hands-On Machine Learning witch Scikit-Learn, Keras & Tensorflow by Aurelien Geron

**Main steps**
1. Look at the big picture
2. Get the data
3. Discover and visualize the data to gain insights
4. Prepare data for ML algorithms
5. Select a model and train it
6. Fine-tune the model
7. Present solution
8. Lunch, monitor, and maintain your system

## Imports

In [2]:
import os

Hint Go thorugh a default check list. This works well for a lot of projects and gives you a starting piont that can be adpted to your needs.

# 1. Look at the big picture

#### 1.1 Frame the problem.
*Theory* <br>
A. What is the business objective? How does your organization aims to benefit from the mode? A good understanding here will help to frame the problem, algorithms to use, performance measurements, and prioritization. Ask what the current solution is, if there is any. This will give you a reference for performance and what to solve. <br> 
B. What system to use? Supervised, unsupervised, reinforcement learning, classification, etc. <br>

*Example* <br>
A. The model should predict if it is worth it to invest in a given area or not (price estimates / market price). Current solution is done manually, time cosuming and costly. The error is ca. 20%. <br>
B. Supervised learning task (given labled data-set), muliple univariant regression task (use of muliple features to predict a numeric value for each district), and plain batch learning (No data stream or need to rapidly adjust data and small enough to fit into memory).  

#### 1.2 Select performance measure
*Theory & Example* <br>
Typical performance measure for a regression system is the Root Mean Square Error. Good estimate for errors with high weight on large errors. 

#### 1.3 Check Assumptions
*Theory* <br>
Check assumptions to avoide mistakes early on. (Communication!)

*Example* <br>
We predict prices but we need to check if the downstream team will not actually convert our estimates into categories.

# 2. Get the data 

#### 2.1 Download

In [3]:
# Download data set - Set variables
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# Download data set, unpack, and store data to local drive.
from functions.get_data import fetch_housing_data
fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH)

In [ ]:
# Load data into memory
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

#### 2.2 Quick look

In [ ]:
# Quick look at data
housing.head()

In [ ]:
# Get statistics of data
housing.describe()

In [ ]:
# Get more quick infos about data shape and types
housing.info()

*Observations:* <br>
Each row represents a district. There are 10 attributes. There are 20,640 instances in the dataset. Some columns (total_bedrooms) do not add up to that number => missing features. All attributes, excpet ocean_proximity, are numeric. We loaded a .csv to we know it must be text attribute if not numeric. Looking ar the first rows of ocean_proximity we see repition, is it a catergorical attribute? *Next step: Look at ocean_proximity*

In [ ]:
# Get more ocean_proximity
housing["ocean_proximity"].value_counts()

*Observation* <br>
5 unique values over all instances => categories.

In [ ]:
# Visualize
housing.hist(bins=50, figsize=(20,15))
plt.show()

*Observations* <br>
1. Median income does not seem to be in USA. Check this => Data set is scaled and caped at 15 for higher income and 0.5 for lower income. These roughly represent 10.000USD this 3 == 30.000USD. *Note* this is a pre-processed attribute. <br>
2. Housing median age and median income are also capped. This might our alogithm to think learn that prices never go beyong that limit. Check if it is necessary to predict behind the limit. IF Ture: either remove those instances from the dataset or gather more date (to inlcude data above the limit). <br>
3. Attributes have different scales.
4. Many histograms are tail-heavy. This makes it harder for some algorithms to detect (the correc) patterns.

#### 2.3 Create a test set

Avoid **data snooping** as our brain is amazing at pattern detection thus might implement a bias early on. Thus we create a test-set early on of roughly 20% of the data set. It is important to always sample / split-off the same data so the model does not get exposed to all data though different runs. This has to be the case even on changes to the dataset. Next, random sampling might introduce a sampling bias. We need to **stratify** the sampling data. Make your you represent each strata accordingly (to few strats=no representation, to many=not enough data per strata).

In [ ]:
# Input: Median income is important featute => Straify data along.
# Balance number of strate vs data/strata
housing["income_cat"] = pd.cut(
    housing["median_income"],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5])

In [ ]:
# Split off stratified (income_cat) test-set
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

Compare the split test set to the original test set.

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

*Observation* <br>
The test set has the same proportions as the main set.

In [ ]:
# Remove the category again  # Artificially added data.
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# 3. Discover and visualization

In [ ]:
# Make a copy of the training set to work with.
housing = strat_train_set.copy()

#### 3.1 Visualize

In [ ]:
# Download the California image
images_path = os.path.join("./", "images")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
filename = "california.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/end_to_end_project/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

In [ ]:
import matplotlib.image as mpimg
california_img=mpimg.imread(os.path.join(images_path, filename))
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                  s=housing['population']/100, label="Population",
                  c="median_house_value", cmap=plt.get_cmap("jet"),
                  colorbar=False, alpha=0.4)
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar(ticks=tick_values/prices.max())
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
plt.show()

#### 3.2 Correlations

Remember, the correlation coefficient only measures linear correlatons and might miss out non linear correlations

In [ ]:
# Calculate pearson-correlation
corr_matrix = housing.corr()

# Correlation example of attributes to median_house_value correlation.
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# Plot pearson correlations via Pandas
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

Investiagate a promising attribute, median-income.

*Observations*
- We can see the 500 000 cap in the median_house_value.
- Seemingly more caps at 480 000, 350 000 and others.
- Clear upwards trend.

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)
plt.axis([0, 16, 0, 550000])
plt.show()

#### 3.3 Attribute combinations

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

*Obervation* We can see that the bedroom to room ration is strong correlated to the median-house-value then total_rooms or total_bedrooms. This is an example of an insight you can gain by just playing the data. This step it not ment to be exaustive but a chance to play through some ideas.

# 4. Preparation

#### 4.1 Cleaning

In [ ]:
# Revert to clean data set & seperate predictors and lables as we want to apply
# different transformations to them.
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

**Dealing with missing features:** <br>
| Get rid of entries with missing feature <br>
| Get rid of attribute <br>
| Set a value

In [ ]:
# Use sklearn simple imputer to fill missing values fields.
# Works only with numerical data, thus apply on a copy without ocean-proximity.
from sklearn.impute import SimpleImputer

# Initi imputer and set strategy.
imputer = SimpleImputer(strategy="median")

# Create copy of purely numerical data.
housing_num = housing.drop("ocean_proximity", axis=1)

# Fit imputer to these data.
imputer.fit(housing_num)

# Transform training set
X = imputer.transform(housing_num)

The imputer is now trained, us it to fill the training set and future data.

**Dealing with cateforical data**
- Algorithms prefer numerical data.

In [ ]:
# OrdinalEncoder encodes catergorical attributes as numericals.
from sklearn.preprocessing import OrdinalEncoder

housing_cat = housing[["ocean_proximity"]]
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

*Problem:* The ML algorithm might assume that close numerical values are similar.
A solution is one-hot-encoding, a sparse matrix with entries as rows and catergory
(numerical) as columns where a 1 marks the category.

In [ ]:
# One hot encoding
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

*Note:* To many categories might lead to performance problmes, evene with sparse
matrices. We could try to find a nother useful numerical feature. i.e. distance
to the ocean in our case.

#### 4.2 Custom Transformers

At one point we will need to write our own transformators. Keep them in keep thier API consisten with the sklearn-API. Simply add 3 methods, transform(), fit(), and fit_transform().

In [ ]:
# Example that adds the adds the combined attributes from before.
from sklearn.base import BaseEstimator, TransformerMixin


# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6


class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    """ Example of a custome transformer.
    
        1 hyperparameter (add_beedrooms_per_room).
        This transformer allows easy add (or not) add_beedrooms_per_room
        to find out if adding this attributes gives some insight.
    """
    
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[
                X,
                rooms_per_household,
                population_per_household,
                bedrooms_per_room
            ]        
        else:
            return np.c_[X, rooms_per_household, population_per_household]

        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

#### 4.3 Feature scaling

Algorithms do not perform well when features are not scaled. We have 2 options, min-max scaling (normalization) and standarization.

- normalization: Values are shifted and rescaled to values between 0 and 1. ((x - min) / max);
- standardization: Subtract the mean (standardized values always have a mean of 0) && devide by the standard-deviation so that the resulting distribution has a unit variance. ((x - mean) / std). Unlike min-max this does not bound values to specific ranges wich can be a problem for some algirithms. However, it is much less affected by outliers. 

#### 4.4 Transformation pipelines
Organize transformation steps execute in the correct order.

In [ ]:
# Pipleline calls fit_transdormer() sequencially on all transformers.
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# Bundle columns transformers (numerical & catergorical).
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

# 5. Select a model

#### 5.1 Try a linear regression model

In [ ]:
# Train the model
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Try it out on a few instances of test-set
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
# Measure performance on whole set with RMSE. NOT ON THE TEST SET!
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(f'Average prediction error: {lin_rmse}')

68k USD error range with a median price of 120k to 265k is not very impressive. The model is underfiting. We can do better. This can mean the features provide not enough information or the model is not good enough. We can improve the model by feeding better features, reduce constrains, and change the model.

#### 5.2 Try another model

**DecisionTreeRegressor**

In [ ]:
# Train the model
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Test the model. NOT ON THE TEST SET!
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

An error of 0. This can not be true. It is likely that the model overfited the data. We can only be sure if we test it on the test-set. However we only want to include it when we a certain about our model to avoide exposing the test data. So we use some training set data for training and some for testing.

#### 5.3 Cross validation

Scikit-Learns K-Fold cross-validation feature splits the code into 10 distinct subsets called folds, then iterates over these folds, use the selected fold as test-set and the others as training set and collects the performance scores.

In [ ]:
# Corss validate the DescisionTreeRegressor using K-Fold-10.
# Expects an utility functions (higher is better) rather than a loss-function.
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

The result is worse then our LinearRegression. *Note* that we can see how accurate the predictions are (std).

**LinearRegression**

In [ ]:
# Cross valdiate LinearRegression
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

**RandomForestRegressor**

In [ ]:
# Train
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Test & Validate
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

Best of the three. However, the score on the training set is lower (better) then on the validation set => overfitting.

# 6. Fine tune the model

**6.1 Grid search**
Grid search different hyperparameter combinations.

In [ ]:
# Try differenct hyperparameter combination for RandomForestRegressor using gird-serach.
# What theses parameters mean will be explained in another chapter.
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {
        'n_estimators': [3, 10, 30],   # We know the answer already,
        'max_features': [2, 4, 8, 16], # but save some time.
    },
    # then try 6 (2×3) combinations with bootstrap set as False
    {
        'bootstrap': [False],
        'n_estimators': [3, 10],
        'max_features': [2, 3, 4]},
  ]

# Init regressor
forest_reg = RandomForestRegressor(random_state=42)

# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(
    forest_reg,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

# Fi
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
# Get best estimator => Hyperparameter setting
param_comb = grid_search.best_params_
cvrs = cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    if params == param_comb: 
        print(np.sqrt(-mean_score), params)

We got a better score then using the default hyperparamter settings. 

*Remember, we can include data preparations steps as hyperparamters.*

**Randomize Search**
When the hyperparamter space is large, randomized search can be more conventient.

**Ensemble methodes**
Bundle the models that perform the best. The group will often perform better then the inidvidual ones. This is especially true if they make different kinds of errors.

**Analyze best models and thier errors**
Some models let you investigate which parameters are important. (Sensitivity?). You may want to drop some of the less important ones.

#### 6.2 Evaluate on test set
- We tweaked our models and think they are good enough to give them a try.
- Nothing special here just remember NOT TO FIT! the models.

In [ ]:
# Get the best estimator
final_model = grid_search.best_estimator_

# Select the data
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# Feed to pipeline
X_test_prepared = full_pipeline.transform(X_test)

# Predict
final_prediction = final_model.predict(X_test_prepared)

# Measure performance
final_mse = mean_squared_error(y_test, final_prediction)
final_rmse = np.sqrt(final_mse)

*Note:* Resist the tempation to fine tune your hyper-parameter to much. This would lead for them to be overtuned for the training set.

You got an estimate how accuarte the model is but you might also want to find out how *precise* it is.

In [ ]:
# Calculate precision of model
from scipy import stats

confidence = 0.95
squared_errors = (final_prediction - y_test) ** 2
np.sqrt(stats.t.interval(
    confidence,
    len(squared_errors) - 1,
    loc=squared_errors.mean(),
    scale=stats.sem(squared_errors)
))

# 7. Present
- What did we learn?
- Assumptions?
- Limitations?
- What worked and what not?
- Document everything.
- Visualization

# 8. Lunch, Monitor, and Maintain your system.

- Create a Web-Api for it. Here a services that handles request-processing and another that only takes in a POST with the input, estimates, and returns the result.
- You need monitoring, the performance. As a rule of thum the performane deteriates over time if not adopted. i.e. changing cameras.
- Regulary collect fresh data and lable it.
- Automate fine-tuning, deploying and monitoring.
- Automate re-traing: Compate old and new model performance and choose the better one.
- Monitor data-input.
- Keep backups of model and data-set.
- Your could create subset of test-set and see how the model performs on different sub-sets. 

# Glossary in order in which they appear during this chapter. 

Pipelines: Sequence of data processing components. They ususally work asynchronously and are connected via the data store. Here each component pulls data from a store, process is and stores the reults in another store.

Notation: <br>

m: number of instances in a dataset. <br>
x^i: feature vector. <br>
X: matrix containing all feature vectors of all dataset instances. <br>
h: sytems prediction function. <br>
RMSE(X,h): cost function. <br>